# Not So Scary Data Science
The purpose of this notebook is to show an intro level approach to the problem, help myself grow as a new data scientist and try provide some simple techniques and guidance to others who may also be just beginning.  At first I was intimidated by the subject of artificial intelligence, but after a few weeks of researching, videos and kernel overview I found out python can make tasks not seem as frightening.  A few notes before getting started: I am new to both python and machine learning, so if I interpret something incorrectly, code something wrong, ect. don't hesitate to tell me.  As mentioned above I am using this notebook to also help myself learn.

Enough of the intro...let's dive into some code

## Imports

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

## Load and Inspect the Data

In [ ]:
train = pd.read_csv("../input/train.csv")

In [ ]:
train.shape

In [ ]:
train.head(5)

In [ ]:
train.tail(5)

In [ ]:
# Check for null values
train.isnull().values.any()

In [ ]:
# Check for any empty string values
for text_id, text, author in zip(train.id, train.text, train.author):
    if text == '' or author == '':
        print(text_id)

In [ ]:
num_total = len(train)
num_eap = len(train.loc[train.author == 'EAP'])
num_hpl = len(train.loc[train.author == 'HPL'])
num_mws = len(train.loc[train.author == 'MWS'])

fig, ax = plt.subplots()
eap, hpl, mws = plt.bar(np.arange(1, 4), [(num_eap/num_total)*100, (num_hpl/num_total)*100, (num_mws/num_total)*100])
ax.set_xticks(np.arange(1, 4))
ax.set_xticklabels(['Edgar Allan Poe', 'H.P. Lovecraft', 'Mary Shelley'])
ax.set_ylim([0, 60])
ax.set_ylabel('Percentage of Entries', fontsize=12)
ax.set_xlabel('Author Name', fontsize=12)
ax.set_title('Percentage of Text Entries by Author')
plt.show()

This looks like a pretty good distribution of authors.

## Split the Data
80% for training, 20% for testing

In [ ]:
feature_col_names = ['text']
predicted_class_name = ['author']
x = train[feature_col_names].values
y = train[predicted_class_name].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)

# Flatten the arrays for later use
x_train = x_train.ravel()
x_test = x_test.ravel()
y_train = y_train.ravel()
y_test = y_test.ravel()

As seen above there are no null or empty values, so no additional data preparation is needed.

## Tokenize and Train an Initial Algorithm - MultinomialNB

In [ ]:
word_vectorizer = TfidfVectorizer()
x_train_word = word_vectorizer.fit_transform(x_train)
x_test_word = word_vectorizer.transform(x_test)

word_clf = MultinomialNB()
word_clf.fit(x_train_word, y_train)

print("Accuracy of word_clf: {0}".format(accuracy_score(y_test, word_clf.predict(x_test_word))))
print("log_loss of word_clf: {0}".format(log_loss(y_test, word_clf.predict_proba(x_test_word))))

Just plug and chug gets an initial accuracy of 81% and a log loss of .6.  Not bad for a first attempt, but both the vectorizer and the classifier has some parameters we can adjust to try and improve our score.

In [ ]:
word_clf = MultinomialNB(alpha=.5)
word_clf.fit(x_train_word, y_train)

print("Accuracy of word_clf with alpha=.5: {0}".format(accuracy_score(y_test, word_clf.predict(x_test_word))))
print("log_loss of word_clf with alpha=.5: {0}".format(log_loss(y_test, word_clf.predict_proba(x_test_word))))

word_clf = MultinomialNB(alpha=.1)
word_clf.fit(x_train_word, y_train)

print("Accuracy of word_clf with alpha=.1: {0}".format(accuracy_score(y_test, word_clf.predict(x_test_word))))
print("log_loss of word_clf with alpha=.1: {0}".format(log_loss(y_test, word_clf.predict_proba(x_test_word))))

Adjusting the alpha is improving the accuracy and log loss, but there must be a better way than manually adjusting the values by hand.  This is where SKLearn's Pipeline and GridSearchCV comes in handy.  It can automate all this testing by hand.

In [ ]:
pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer()), ('mnb', MultinomialNB())])
    
parameters = {'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
              'mnb__alpha': [.1, .05, .005]}

# For more updates on progress change verbose to something like 10
grid = GridSearchCV(pipeline, parameters, scoring='accuracy', cv=10, verbose=1, n_jobs=-1)

grid.fit(x_train, y_train)

In [ ]:
print('Best score: {0}'.format(grid.best_score_))
print('Best parameters: {0}'.format(grid.best_params_))

Other things that are testable: a mixture of different vectorizers, classifiers and all of their settings.  So far from all my testing I found out using TfidfVectorizer with the MultinomialNB is best.  Now, time to retrain with these values and output the performance.

In [ ]:
word_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
x_train_word = word_vectorizer.fit_transform(x_train)
x_test_word = word_vectorizer.transform(x_test)

word_clf = MultinomialNB(alpha=.05)
word_clf.fit(x_train_word, y_train)

word_clf_pred_proba = word_clf.predict_proba(x_test_word)

print("Accuracy of word_clf: {0}".format(accuracy_score(y_test, word_clf.predict(x_test_word))))
print("log_loss of word_clf: {0}".format(log_loss(y_test, word_clf_pred_proba)))

Looks like we bumped up the accuracy and log loss even further, excellent.  Let's try something else.  By default the TfidfVectorizer tokenizes by words, let's try characters instead.  Note: due to the increased amount of time character analysis takes I am skipping the Pipeline testing and returning the result I found to be best so far.

In [ ]:
char_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 9))
x_train_char = char_vectorizer.fit_transform(x_train)
x_test_char = char_vectorizer.transform(x_test)

char_clf = MultinomialNB(alpha=.005)
char_clf.fit(x_train_char, y_train)

char_clf_pred_proba = char_clf.predict_proba(x_test_char)

print("Accuracy of char_clf: {0}".format(accuracy_score(y_test, char_clf.predict(x_test_char))))
print("log_loss of char_clf: {0}".format(log_loss(y_test, char_clf_pred_proba)))

I found this pretty interesting, I increased the accuracy, but at the expense of log loss.  If I understand this correctly, the character analyzer is predicting correctly more times than the word analyzer, but it is less confident on each of the results.

Now that two different classifiers let's look at some confusion matrices.

In [ ]:
# From http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    #else:
        #print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
word_conf_mat = confusion_matrix(y_test, word_clf.predict(x_test_word))
plot_confusion_matrix(word_conf_mat, classes=['EAP', 'HPL', 'MWS'], title='Author Confusion Matrix - Word Classifier')

In [ ]:
char_conf_mat = confusion_matrix(y_test, char_clf.predict(x_test_char))
plot_confusion_matrix(char_conf_mat, classes=['EAP', 'HPL', 'MWS'], title='Author Confusion Matrix - Character Classifier')

Looks like each is better at predicting different aspects, what happens if we try to combine both classifiers (ensembling)?

In [ ]:
combined_proba = []
combined_predictions = []
for w, c in zip(word_clf_pred_proba, char_clf_pred_proba):
    prob = (w[0]+c[0])/2, (w[1]+c[1])/2, (w[2]+c[2])/2
    combined_proba.append(prob)
    if prob[0] > prob[1] and prob[0] > prob[2]:
        combined_predictions.append('EAP')
    elif prob[1] > prob[2]:
        combined_predictions.append('HPL')
    else:
        combined_predictions.append('MWS')
        
print("Accuracy of combined classifiers: {0}".format(accuracy_score(y_test, combined_predictions)))
print("log_loss of combined classifiers: {0}".format(log_loss(y_test, combined_proba)))

In [ ]:
combined_conf_mat = confusion_matrix(y_test, combined_predictions)
plot_confusion_matrix(combined_conf_mat, classes=['EAP', 'HPL', 'MWS'], title='Author Confusion Matrix - Combined Classifiers')

## The End....Maybe?
As I mentioned before this notebook is designed to both help others and myself learn.  If you have any questions, comments, suggestions or improvements please mention them.